Basic Evaluation Workflow
==========================
This notebook demonstrates evaluating a single downscaled model.

In [1]:
# Setup

import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

import matplotlib.pyplot as plt
import xarray as xr

from climate_eval import io, metrics, plotting
from climate_eval.core import EvaluationResults
from climate_eval.utils import standardize_dimension_names

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')

print("ClimatExEval loaded successfully!")

ClimatExEval loaded successfully!


In [2]:
# Load configuration from YAML file.

config = io.load_config('../config.yaml')
print(f"\nProject: {config.project_name}")
print(f"Description: {config.description}")
print(f"Metrics to compute: {config.metrics.all_metrics()}")

🔧 Loading configuration from ../config.yaml
📊 Dataset configured: /home/sbeairsto/data-sbeairsto/van_isle_128/trained_generators/hr_inf_generator_bs_64_ns_6_lr_00025_drif_pen_uas.zarr (zarr)
📊 Dataset configured: /home/sbeairsto/data-sbeairsto/van_isle_128/uas_validation_hr.zarr (zarr)
📈 Metrics configured: 4 total
⚡ Dask enabled with 4 workers
💾 Output directory: results
🌍 Evaluation project: uas_evaluation
   Evaluation of GAN-downscaled uas winds



Project: uas_evaluation
Description: Evaluation of GAN-downscaled uas winds
Metrics to compute: ['bias', 'quantile_comparison', 'spatial_correlation', 'temporal_autocorrelation']


In [3]:
# Load predicted and reference datasets.

pred_dataset = io.load_dataset(config.data_predicted, dataset_type="predicted")
ref_dataset = io.load_dataset(config.data_reference, dataset_type="reference")

print(f"\nPredicted: {pred_dataset}")
print(f"Reference: {ref_dataset}")

   Found 5 realizations
   Selecting first realization
📥 Loaded predicted data: {'time': 8760, 'rlon': 128, 'rlat': 128}
   No 'realization' dimension found
📥 Loaded reference data: {'time': 8760, 'rlat': 128, 'rlon': 128}



Predicted: LoadedDataset(type=predicted, variables=['uas'], shape={'time': 8760, 'rlon': 128, 'rlat': 128})
Reference: LoadedDataset(type=reference, variables=['uas'], shape={'time': 8760, 'rlat': 128, 'rlon': 128})


In [4]:
# Preprocess

# Standardize dimension names (rlat/rlon -> lat/lon)
pred_dataset.data = standardize_dimension_names(pred_dataset.data)
ref_dataset.data = standardize_dimension_names(ref_dataset.data)

# Subset to domain of interest
pred_dataset.data = io.subset_domain(pred_dataset.data, config.domain)
ref_dataset.data = io.subset_domain(ref_dataset.data, config.domain)

# Align grids and variable names
pred_dataset, ref_dataset = io.align_datasets(pred_dataset, ref_dataset)

print(f"\nAfter preprocessing:")
print(f"Predicted: {dict(pred_dataset.data.sizes)}")
print(f"Reference: {dict(ref_dataset.data.sizes)}")

Renaming dimensions: {'rlat': 'lat', 'rlon': 'lon'}
Renaming dimensions: {'rlat': 'lat', 'rlon': 'lon'}
🔄 Aligning datasets...
   Renamed variables: {'uas': 'uas'}
   Common variables: ['uas']
   Grids already aligned



After preprocessing:
Predicted: {'time': 8760, 'lon': 128, 'lat': 128}
Reference: {'time': 8760, 'lat': 128, 'lon': 128}


In [5]:
# Create results container for storing all metrics.

results = EvaluationResults(config=config)
print(f"\nResults container initialized for: {results.config.project_name}")


Results container initialized for: uas_evaluation


In [6]:
# Choose variable to evaluate.

# Change this to evaluate different variables
variable = 'uas'

pred_var = pred_dataset[variable]
ref_var = ref_dataset[variable]

print(f"\nEvaluating variable: {variable}")
print(f"Shape: {pred_var.shape}")


Evaluating variable: uas
Shape: (8760, 128, 128)


In [7]:
# Compute mean bias field.

print("\n" + "="*50)
print("MARGINAL METRICS")
print("="*50)

# Compute bias (time mean)
bias_field = metrics.bias(pred_var, ref_var, dim='time')
results.add_result(f'{variable}_bias', bias_field)
print(f"Mean bias: {float(bias_field.mean()):.4f}")

/home/sbeairsto/projects/ClimatExEval/.eval_venv/lib/python3.12/site-packages/dask/array/core.py:5198: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(



MARGINAL METRICS


✅ Computed metric: uas_bias


Mean bias: 0.5665


In [8]:
# Compare quantile distributions.

quantile_results = metrics.quantile_comparison(pred_var, ref_var)  # Renamed variable
results.add_result(f'{variable}_quantiles', quantile_results)
print(f"Quantiles computed: {list(quantile_results.coords['quantile'].values)}")

✅ Computed metric: uas_quantiles


Quantiles computed: [np.float64(0.01), np.float64(0.05), np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.95), np.float64(0.99)]


In [9]:
# Compute spatial correlation field.

print("\n" + "="*50)
print("SPATIAL METRICS")
print("="*50)

spatial_corr = metrics.spatial_correlation(pred_var, ref_var)
results.add_result(f'{variable}_spatial_correlation', spatial_corr)
print(f"Mean spatial correlation: {float(spatial_corr.mean()):.4f}")


SPATIAL METRICS


/home/sbeairsto/projects/ClimatExEval/.eval_venv/lib/python3.12/site-packages/dask/array/core.py:5198: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(
✅ Computed metric: uas_spatial_correlation


Mean spatial correlation: 0.2805


In [ ]:
# Compute temporal autocorrelation.

print("\n" + "="*50)
print("TEMPORAL METRICS")
print("="*50)

# Compute ACF for predicted data
pred_acf = metrics.temporal_autocorrelation(pred_var, max_lag=30)
results.add_result(f'{variable}_pred_acf', pred_acf)

# Compute ACF for reference data
ref_acf = metrics.temporal_autocorrelation(ref_var, max_lag=30)
results.add_result(f'{variable}_ref_acf', ref_acf)

print(f"ACF computed with {len(pred_acf.lag)} lags")

   Loading data into memory for ACF computation...



TEMPORAL METRICS


In [ ]:
# Visualize distribution comparison.

fig, ax = plotting.distributions.plot_histogram_comparison(
    pred_var, ref_var,
    title=f'{variable.title()} Distribution Comparison',
    bins=50
)
plt.show()

In [ ]:
# Compute max value for color scale
bias_max = float(abs(bias_field).max().compute())  # Added .compute()

fig, ax = plotting.maps.plot_spatial_field(
    bias_field,
    title=f'{variable.title()} Time-Mean Bias',
    cmap='RdBu_r',
    vmin=-bias_max,
    vmax=bias_max
)
plt.show()

In [ ]:
# Compare temporal autocorrelation.

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot predicted ACF
plt.subplot(1, 2, 1)
pred_acf.mean(dim=['lat', 'lon']).plot(marker='o', label='Predicted')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Lag (days)')
plt.ylabel('Autocorrelation')
plt.title('Predicted ACF')
plt.grid(True, alpha=0.3)

# Plot reference ACF
plt.subplot(1, 2, 2)
ref_acf.mean(dim=['lat', 'lon']).plot(marker='s', label='Reference', color='C1')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Lag (days)')
plt.ylabel('Autocorrelation')
plt.title('Reference ACF')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Display summary of all scalar metrics

print("\n" + "="*50)
print("SUMMARY")
print("="*50)

summary = results.summary()
if summary:
    for metric_name, value in summary.items():
        print(f"{metric_name}: {value:.6f}")
else:
    print("No scalar metrics computed.")

print(f"\nTotal metrics computed: {len(results.results)}")

In [ ]:
# Save all results to disk.

io.save_results(results, name=variable)
print(f"\n✅ Results saved to: {config.output.dir / config.project_name}")